# **Loading Dataset into DataFrame**

In [1]:
import pandas as pd
import sqlite3
import os

def load_dataset(file_path):
    if not os.path.exists(file_path):
        print(f"Error: File does not exist.")
        return None

    try:
        print(f"Loading dataset from into DataFrame...")
        df = pd.read_csv(file_path)
        print("Finished loading dataset.")
        return df
    except pd.errors.EmptyDataError:
        print("Error: The file is empty.")
    except pd.errors.ParserError:
        print("Error: There was an issue parsing the file.")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
    return None

# Path to the dataset
file_path = 'job_descriptions.csv'

# Load the dataset
df = load_dataset(file_path)


Error: File does not exist.


In [2]:

try:
    # Connect to the SQLite database
    conn = sqlite3.connect('job.db')

    # Replace spaces with underscores in column names
    columns = df.columns.tolist()
    df.columns = df.columns.str.replace(' ', '_')

    # Create an FTS5 table with the cleaned-up column names
    columns_definitions = ', '.join(df.columns)
    conn.execute(f'''
    CREATE VIRTUAL TABLE IF NOT EXISTS my_table USING fts5({columns_definitions});
    ''')

    # Insert DataFrame data into the FTS5 table
    df.to_sql('my_table', conn, if_exists='replace', index=False)
    print("Data successfully inserted into FTS5 table.")
    
except sqlite3.Error as e:
    print(f"SQLite error: {e}")
except Exception as e:
    print(f"Unexpected error: {e}")
finally:
    # Close the connection
    if conn:
        conn.close()
        print("Connection closed.")


Unexpected error: 'NoneType' object has no attribute 'columns'
Connection closed.


In [3]:
def search_database(conn, columns, keywords, limit, match_all=False):
    """ 
    Search the SQLite database using Full-Text Search (FTS5).
    
    :param conn: SQLite connection object 
    :param columns: List of column names to search 
    :param keywords: List of keywords to search for 
    :param limit: Number of rows to return 
    :param match_all: If True, all columns must match the keywords (AND). Default is OR.
    :return: List of matching rows 
    """
    
    if not keywords or not columns:
        return []  # Return empty if no keywords or columns provided

    # Create a space-separated string for the keywords
    keywords_query = ' '.join(keywords)

    # Build the MATCH clause, using AND or OR depending on match_all
    operator = ' AND ' if match_all else ' OR '
    columns_match = operator.join([f"{col} MATCH ?" for col in columns])

    # Prepare the SQL query
    query = f"SELECT * FROM my_table WHERE {columns_match} LIMIT ?;"

    # Prepare parameters: the keywords query repeated for each column plus the limit
    parameters = [keywords_query] * len(columns) + [limit]

    # Execute the query and fetch the results
    try:
        cursor = conn.execute(query, parameters)
        results = cursor.fetchall()
        return results
    except sqlite3.Error as e:
        print(f"SQLite error: {e}")
        return []



In [4]:
columns_to_search = ['Job_Description', "skills"]  # All columns
keywords = ['Python', 'Java']  # Replace with the keywords you want to search for

results = search_database(conn, columns=columns_to_search, keywords=keywords, limit=10, match_all=True)

# Display the results
for row in results:
    print(row)

SQLite error: Cannot operate on a closed database.
